# TD2 - Développement de Processus d'Analyse de Données et de Feature Engineering avec Spark SQL

Ce notebook couvre les concepts clés du développement de processus d'analyse de données et de feature engineering avec Spark SQL, notamment les DataFrames,les types de données, les opérations communes, les agrégations, les jointures et les fonctions de fenêtre.

---

## **Table des Matières**

1. Spark SQL
2. DataFrame
3. DataFrame VS RDD
4. DataTypes
5. Opérations Communes
6. Agrégations (GroupBy)
7. Fenêtres (Windows)
8. Jointures


In [1]:
# Installer Pyspark
!pip install pyspark

In [2]:
from datetime import datetime

from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DecimalType, FloatType
from pyspark.sql import functions as F

In [3]:
# Initialiser SparkSession
spark = SparkSession.builder.appName("TD2").getOrCreate()

# Spark SQL
Spark SQL est un module de haut niveau de Spark permettant d'interroger et manipuler des données structurées en utilisant SQL ou l’API DataFrame de Spark en offrant des optimisations puissantes.

# DataFrame

Les DataFrames Spark sont inspirés des DataFrames de pandas en termes de structure et de format cependant les Spark DataFrames sont distribués contrairement à pandas aux dataframes pandas en mémoire avec des colonnes nommées et des schémas définis, où chaque colonne possède un type de données spécifique (integer, string, array, date, etc.). Visuellement, un DataFrame Spark apparaît comme une table.

Les DataFrames sont conçus comme une couche au-dessus des RDDs, ajoutant des métadonnées supplémentaires grâce à leur format tabulaire, ce qui facilite les opérations de traitement de données.

In [4]:
# Créer les données
data = [
    (1, "Le Petit Prince", "Antoine de Saint-Exupéry", datetime(1943,4,6), 12.50, "Fiction"),
    (2, "1984", "George Orwell", datetime(1949,6,8), 15.00, "Dystopie"),
    (3, "L'Étranger", "Albert Camus", datetime(1942,5,19), 10.00, "Philosophie"),
    (4, "Harry Potter à l'école des sorciers", "J.K. Rowling", datetime(1997,6,26), 20.00, "Fantastique"),
    (5, "Les Misérables", "Victor Hugo", datetime(1862,4,3), 18.00, "Drame"),
    (6, "Le Seigneur des Anneaux", "J.R.R. Tolkien", datetime(1954,7,29), 25.00, "Fantastique"),
    (7, "To Kill a Mockingbird", "Harper Lee", datetime(1960,7,11), 14.00, "Fiction"),
    (8, "Crime et Châtiment", "Fiodor Dostoïevski", datetime(1866,1,1), 16.50, "Psychologique"),
    (9, "Pride and Prejudice", "Jane Austen", datetime(1813,1,28), 13.00, "Romance"),
    (10, "Moby Dick", "Herman Melville", datetime(1851,10,18), 17.00, "Aventure")
]
# Définir le schéma
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("Livre", StringType(), False),
    StructField("Auteur", StringType(), False),
    StructField("Date de Parution", DateType(), False),
    StructField("Prix de Vente (€)", FloatType(), False),
    StructField("Genre", StringType(), False)
])
# Créer le DataFrame avec le schéma
df_livres = spark.createDataFrame(data, schema)
df_livres.toPandas()

,id,Livre,Auteur,Date de Parution,Prix de Vente (€),Genre
0,1,Le Petit Prince,Antoine de Saint-Exupéry,1943-04-06,12.5,Fiction
1,2,1984,George Orwell,1949-06-08,15.0,Dystopie
2,3,L'Étranger,Albert Camus,1942-05-19,10.0,Philosophie
3,4,Harry Potter à l'école des sorciers,J.K. Rowling,1997-06-26,20.0,Fantastique
4,5,Les Misérables,Victor Hugo,1862-04-03,18.0,Drame
5,6,Le Seigneur des Anneaux,J.R.R. Tolkien,1954-07-29,25.0,Fantastique
6,7,To Kill a Mockingbird,Harper Lee,1960-07-11,14.0,Fiction
7,8,Crime et Châtiment,Fiodor Dostoïevski,1866-01-01,16.5,Psychologique
8,9,Pride and Prejudice,Jane Austen,1813-01-28,13.0,Romance
9,10,Moby Dick,Herman Melville,1851-10-18,17.0,Aventure




# RDD vs DataFrame

| **Caractéristique** | **RDD (Resilient Distributed Dataset)** | **DataFrame** |
|---------------------|-------------------------------------------|----------------|
| **Schéma** | Données non structurées ou semi-structurées | Schéma défini avec colonnes nommées et typées|
| **Performance** | Moins optimisé | Hautement optimisé grâce au Catalyst Optimizer |
| **API** | Bas niveau, nécessite plus de code pour les opérations courantes | Haut niveau, syntaxe concise et expressive similaire à SQL |
| **Support SQL** | Non pris en charge | Prise en charge des requêtes SQL |

### Quand utiliser l’un ou l’autre

- **RDD** : Utilisez les RDDs lorsque vous devez manipuler des données non structurées ou semi-structurées, ou appliquer des transformations de bas niveau.
- **DataFrame** : Préférez les DataFrames pour le traitement de données structuréee et les opérations SQL.

# Types de Données (DataTypes) dans les DataFrames PySpark

Les types de données dans PySpark sont organisés en différentes catégories. Voici une présentation des principaux types de données avec leurs descriptions et des exemples.

### 1. Types Numériques

| **Type PySpark** | **Description** | **Exemple** |
|------------------|-----------------|-------------|
| `ByteType` | Entier sur 1 octet (-128 à 127) | `100` |
| `ShortType` | Entier sur 2 octets (-32 768 à 32 767) | `1000` |
| `IntegerType` | Entier sur 4 octets (-2,147,483,648 à 2,147,483,647) | `100000` |
| `LongType` | Entier sur 8 octets (-9,223,372,036,854,775,808 à 9,223,372,036,854,775,807) | `10000000000` |
| `FloatType` | Nombre à virgule flottante simple précision | `3.14` |
| `DoubleType` | Nombre à virgule flottante double précision | `3.1415926535` |
| `DecimalType(precision, scale)` | Nombre décimal avec précision et échelle spécifiques | `DecimalType(10, 2)` pour `12345678.90` |

### 2. Type Chaîne

| **Type PySpark** | **Description** | **Exemple** |
|------------------|-----------------|-------------|
| `StringType` | Chaîne de caractères | `"Bonjour"`, `"PySpark"` |

### 3. Type Booléen

| **Type PySpark** | **Description** | **Exemple** |
|------------------|-----------------|-------------|
| `BooleanType` | Valeur booléenne (`True` ou `False`) | `True`, `False` |

### 4. Types Date et Heure

| **Type PySpark** | **Description** | **Exemple** |
|------------------|-----------------|-------------|
| `DateType` | Date sans information de temps | `"2023-10-01"` |
| `TimestampType` | Horodatage avec date et heure | `"2023-10-01 12:34:56"` |

### 5. Types Complexes

| **Type PySpark** | **Description** | **Exemple** |
|------------------|-----------------|-------------|
| `ArrayType(elementType)` | Tableau d'éléments du type spécifié | `["a", "b", "c"]` |
| `MapType(keyType, valueType)` | Map avec des clés et des valeurs de types spécifiés | `{"a": 1, "b": 2}` |

In [6]:
df = spark.read.csv("data td1.csv", inferSchema=False, header=True)
df.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_categorie: string (nullable = true)
 |-- boutique: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- date: string (nullable = true)



# Opérations Communes

## Select

L'utilisation la plus basique de select consiste à extraire une ou plusieurs colonnes d'un DataFrame existant. Cela permet de réduire la taille du DataFrame en ne conservant que les colonnes d'intérêt, facilitant ainsi les analyses ultérieures.

In [7]:
df.select("boutique", "date").limit(5).toPandas()

,boutique,date
0,Boutique_1,2024-01-14
1,Boutique_21,2024-02-07
2,Boutique_30,2023-12-06
3,Boutique_41,2024-06-17
4,Boutique_6,2024-06-12


Au-delà de la simple sélection de colonnes, l'opérateur select permet d'effectuer diverses manipulations avancées sur les données :



* Renommer des Colonnes
* Caster des Colonnes
* Effectuer des Calculs sur les Colonnes
* Utiliser des Fonctions SQL (comme min, lower, etc.)


In [8]:
# Renommage de la colonne boutique
df.select(F.col("boutique").alias("BOUTIQUE"), "date").limit(10).toPandas()

,BOUTIQUE,date
0,Boutique_1,2024-01-14
1,Boutique_21,2024-02-07
2,Boutique_30,2023-12-06
3,Boutique_41,2024-06-17
4,Boutique_6,2024-06-12
5,Boutique_25,2024-05-05
6,Boutique_47,2023-10-05
7,Boutique_35,2024-03-27
8,Boutique_18,2024-09-05
9,Boutique_27,2024-09-08


In [10]:
df_prepared = (
    df
    .select(
        # Convertir les colonnes ci-dessous au bon type
        F.col('transaction_id').cast("bigint"),
        F.col('customer_id').cast("bigint"),
        F.col('product_id').cast("bigint"),
        # Transformer les valeurs des colonnes ci-dessous en minuscule
        F.lower('product_categorie').alias("product_categorie"),
        F.lower('boutique').alias("boutique"),
        F.col("amount").cast("float"),
        # Convertir la date au bon format
        F.to_date("date", "yyyy-MM-dd").alias("date")
    )
  )

df_prepared.limit(10).toPandas()

,transaction_id,customer_id,product_id,product_categorie,boutique,amount,date
0,1,1662,752,catégorie_50,boutique_1,50.959999,2024-01-14
1,2,2722,386,catégorie_37,boutique_21,182.740005,2024-02-07
2,3,2298,809,catégorie_89,boutique_30,217.600006,2023-12-06
3,4,1398,953,catégorie_33,boutique_41,401.239990,2024-06-17
4,5,4290,90,catégorie_73,boutique_6,5.860000,2024-06-12
5,6,98,305,catégorie_57,boutique_25,444.040009,2024-05-05
6,7,1166,737,catégorie_77,boutique_47,15.180000,2023-10-05
7,8,4322,116,catégorie_50,boutique_35,129.789993,2024-03-27
8,9,3391,411,catégorie_42,boutique_18,439.390015,2024-09-05
9,10,2154,537,catégorie_27,boutique_27,345.859985,2024-09-08


In [11]:
# Calcul de quelques stats
df_prepared.select(F.sum("amount"), F.avg("amount"), F.min("date")).toPandas()

,sum(amount),avg(amount),min(date)
0,2.525663e+07,252.566268,2023-09-20


# WithColumn

L'opérateur withColumn sert principalement à :

* Calculer des nouvelles colonnes
* Transformation des colonnes existantes
* Appliquer des fonctions

In [12]:
(
    df_prepared
    .withColumn("month", F.month("date"))
    .withColumn("prix avec tva", F.col("amount") * 1.2)
    .limit(10)
    .toPandas()
)

,transaction_id,customer_id,product_id,product_categorie,boutique,amount,date,month,prix avec tva
0,1,1662,752,catégorie_50,boutique_1,50.959999,2024-01-14,1,61.151999
1,2,2722,386,catégorie_37,boutique_21,182.740005,2024-02-07,2,219.288007
2,3,2298,809,catégorie_89,boutique_30,217.600006,2023-12-06,12,261.120007
3,4,1398,953,catégorie_33,boutique_41,401.239990,2024-06-17,6,481.487988
4,5,4290,90,catégorie_73,boutique_6,5.860000,2024-06-12,6,7.032000
5,6,98,305,catégorie_57,boutique_25,444.040009,2024-05-05,5,532.848010
6,7,1166,737,catégorie_77,boutique_47,15.180000,2023-10-05,10,18.216000
7,8,4322,116,catégorie_50,boutique_35,129.789993,2024-03-27,3,155.747992
8,9,3391,411,catégorie_42,boutique_18,439.390015,2024-09-05,9,527.268018
9,10,2154,537,catégorie_27,boutique_27,345.859985,2024-09-08,9,415.031982


# groupby

L’opération groupBy est utilisée pour regrouper les données d’un DataFrame en fonction d’une ou plusieurs colonnes clés. Cela permet d’appliquer des fonctions d’agrégation (comme sum, count, avg, etc.) à chaque groupe distinct

In [13]:
(
    df_prepared
    .groupby("boutique")
    .agg(
        F.min("date").alias("alias_première transaction"),
        F.sum("amount").alias("ca"),
        F.avg("amount").alias("ca_moyen"),
    )
    .limit(10)
    .toPandas()
)

,boutique,alias_première transaction,ca,ca_moyen
0,boutique_10,2023-09-20,520737.079776,254.142059
1,boutique_11,2023-09-20,509826.979951,252.389594
2,boutique_18,2023-09-20,503300.440272,253.424189
3,boutique_12,2023-09-20,508475.229559,254.492107
4,boutique_6,2023-09-20,501729.100201,249.492342
5,boutique_5,2023-09-20,504398.099643,250.819542
6,boutique_13,2023-09-20,500468.540405,244.130995
7,boutique_4,2023-09-21,525000.139859,252.525320
8,boutique_49,2023-09-20,496398.460381,254.824672
9,boutique_40,2023-09-20,508567.060093,253.018438


# Window Function

Les fonctions de fenêtre (window functions) dans PySpark permettent d’effectuer des calculs sur un ensemble de lignes liées à la ligne courante et définies par une ou plusieurs colonnées clés, sans réduire le jeu de données comme le ferait un groupBy. Elles sont essentielles pour des opérations telles que le classement, le calcul de moyennes mobiles, les différences entre lignes, etc.

In [14]:
# Création du DataFrame
data = [
    ('Ventes', 'Alice', 7000),
    ('Ventes', 'Bob', 6000),
    ('Ventes', 'Charlie', 6000),
    ('Ventes', 'Dave', 5000),
    ('RH', 'Eve', 6500),
    ('RH', 'Frank', 6500),
    ('RH', 'Grace', 5500)
]

columns = ['Département', 'Employé', 'Salaire']
df_employe = spark.createDataFrame(data, columns)

# Spécification des fenêtres
w_departement_salaire = Window.partitionBy('Département').orderBy(F.desc("Salaire"))
w_departement = Window.partitionBy('Département')

# Application des fonctions
(
    df_employe
    .withColumn('row_number', F.row_number().over(w_departement_salaire))
    .withColumn('rank', F.rank().over(w_departement_salaire))
    .withColumn('dense_rank', F.dense_rank().over(w_departement_salaire))
    .withColumn('salaire_prec', F.lag('Salaire', 1).over(w_departement_salaire))
    .withColumn('salaire_suivant', F.lead('Salaire', 1).over(w_departement_salaire))

    .withColumn('salaire_par_departement', F.sum("Salaire").over(w_departement))

    .orderBy("Département", F.desc("Salaire"))
    .toPandas()
)


,Département,Employé,Salaire,row_number,rank,dense_rank,salaire_prec,salaire_suivant,salaire_par_departement
0,RH,Eve,6500,1,1,1,NaN,6500.0,18500
1,RH,Frank,6500,2,1,1,6500.0,5500.0,18500
2,RH,Grace,5500,3,3,2,6500.0,NaN,18500
3,Ventes,Alice,7000,1,1,1,NaN,6000.0,24000
4,Ventes,Bob,6000,2,2,2,7000.0,6000.0,24000
5,Ventes,Charlie,6000,3,2,2,6000.0,5000.0,24000
6,Ventes,Dave,5000,4,4,3,6000.0,NaN,24000


* row_number : Numérote chaque employé séquentiellement dans chaque département.
* rank : Attribue le même rang aux ex æquo, mais saute les rangs suivants.
* dense_rank : Comme rank, mais ne saute pas de rangs après les ex æquo.
* lag : Affiche le salaire de l’employé précédent par rapport à ligne courante dans le département.
* lead : Affiche le salaire de l’employé suivant par rapport à ligne courante dans le département.

## Jointures

Les jointures permettent de combiner des lignes de deux DataFrames basés sur une ou plusieurs colonnes communes.

PySpark offre une variété de types de jointures pour répondre à différents besoins d’analyse de données :
* Left Join : Renvoie tous les lignes de la table à gauche
* Right Join : Renvoie tous les lignes de la table à droite
* Inner Join : Renvoie l'intersection des clès de jointures des deux tables
* Cross Join : Renvoie toutes les combinaisons possibles

In [15]:
# Création des DataFrames
data_etudiants = [
    (1, 'Alice', 20),
    (2, 'Bob', 22),
    (3, 'Charlie', 23),
    (4, 'David', 21),
    (5, 'Eve', 22)
]
columns_etudiants = ['Étudiant_ID', 'Nom', 'Âge']

df_etudiants = spark.createDataFrame(data_etudiants, columns_etudiants)

data_cours = [
    (1, 'Mathématiques', 85),
    (2, 'Physique', 78),
    (3, 'Chimie', 92),
    (1, 'Physique', 88),
    (2, 'Mathématiques', 76),
    (6, 'Informatique', 90)
]
columns_cours = ['Étudiant_ID', 'Cours', 'Note']

df_cours = spark.createDataFrame(data_cours, columns_cours)

In [16]:
df_etudiants.toPandas()

,Étudiant_ID,Nom,Âge
0,1,Alice,20
1,2,Bob,22
2,3,Charlie,23
3,4,David,21
4,5,Eve,22


In [17]:
df_cours.toPandas()

,Étudiant_ID,Cours,Note
0,1,Mathématiques,85
1,2,Physique,78
2,3,Chimie,92
3,1,Physique,88
4,2,Mathématiques,76
5,6,Informatique,90


In [18]:
df_left = (
    df_etudiants
    .join(
        df_cours,
        on="Étudiant_ID",
        how="left"
    )
)
df_left.toPandas()

,Étudiant_ID,Nom,Âge,Cours,Note
0,1,Alice,20,Physique,88.0
1,1,Alice,20,Mathématiques,85.0
2,2,Bob,22,Mathématiques,76.0
3,2,Bob,22,Physique,78.0
4,5,Eve,22,None,NaN
5,3,Charlie,23,Chimie,92.0
6,4,David,21,None,NaN


In [19]:
df_right = (
    df_etudiants
    .join(
        df_cours,
        on="Étudiant_ID",
        how="right"
    )
)
df_right.toPandas()

,Étudiant_ID,Nom,Âge,Cours,Note
0,1,Alice,20.0,Mathématiques,85
1,3,Charlie,23.0,Chimie,92
2,2,Bob,22.0,Physique,78
3,6,None,NaN,Informatique,90
4,1,Alice,20.0,Physique,88
5,2,Bob,22.0,Mathématiques,76


In [20]:
df_inner = (
    df_etudiants
    .join(
        df_cours,
        on="Étudiant_ID",
        how="inner"
    )
)
df_inner.toPandas()

,Étudiant_ID,Nom,Âge,Cours,Note
0,1,Alice,20,Mathématiques,85
1,1,Alice,20,Physique,88
2,2,Bob,22,Physique,78
3,2,Bob,22,Mathématiques,76
4,3,Charlie,23,Chimie,92
